# Cardiovascular Disease Prediction using Linear Regression

## Mission Statement
To mitigate labor shortages and boost productivity by leveraging predictive analytics to identify and prevent cardio-related diseases early, ensuring a healthier workforce and a more resilient economy.

## Dataset Features:
- age: Age in years
- gender: Gender (1 = female, 2 = male)
- height: Height in cm
- weight: Weight in kg
- ap_hi: Systolic blood pressure
- ap_lo: Diastolic blood pressure
- cholesterol: Cholesterol level (1 = normal, 2 = above normal, 3 = well above normal)
- gluc: Glucose level (1 = normal, 2 = above normal, 3 = well above normal)
- smoke: Smoking (0 = no, 1 = yes)
- alco: Alcohol intake (0 = no, 1 = yes)
- active: Physical activity (0 = no, 1 = yes)
- cardio: Cardiovascular disease (0 = no, 1 = yes) - TARGET VARIABLE

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## A. Data Visualization and Feature Engineering

In [ ]:
# Load the dataset
df = pd.read_csv('cardio_base.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Basic statistics
print("\nBasic Statistics:")
print(df.describe())

In [ ]:
# Data Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution of target variable
axes[0, 0].pie(df['cardio'].value_counts(), labels=['No Disease', 'Disease'], autopct='%1.1f%%')
axes[0, 0].set_title('Distribution of Cardiovascular Disease')

# Age distribution by cardio disease
sns.boxplot(data=df, x='cardio', y='age', ax=axes[0, 1])
axes[0, 1].set_title('Age Distribution by Cardiovascular Disease')
axes[0, 1].set_xlabel('Cardio Disease (0=No, 1=Yes)')

# Blood pressure relationship
sns.scatterplot(data=df, x='ap_hi', y='ap_lo', hue='cardio', ax=axes[1, 0])
axes[1, 0].set_title('Blood Pressure Relationship')
axes[1, 0].set_xlabel('Systolic BP')
axes[1, 0].set_ylabel('Diastolic BP')

# BMI calculation and distribution
df['bmi'] = df['weight'] / (df['height'] / 100) ** 2
sns.boxplot(data=df, x='cardio', y='bmi', ax=axes[1, 1])
axes[1, 1].set_title('BMI Distribution by Cardiovascular Disease')
axes[1, 1].set_xlabel('Cardio Disease (0=No, 1=Yes)')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix of All Features')
plt.show()

# Feature importance based on correlation with target
target_corr = abs(correlation_matrix['cardio']).sort_values(ascending=False)
print("\nFeature Correlation with Target (Cardiovascular Disease):")
print(target_corr)

In [ ]:
# Feature Engineering
print("Original features:", df.columns.tolist())

# Create additional features
df['pulse_pressure'] = df['ap_hi'] - df['ap_lo']  # Pulse pressure
df['age_risk'] = (df['age'] > 50).astype(int)  # Age risk factor
df['bp_risk'] = ((df['ap_hi'] > 140) | (df['ap_lo'] > 90)).astype(int)  # High BP risk
df['lifestyle_risk'] = df['smoke'] + df['alco'] - df['active']  # Lifestyle risk score

print("\nNew features added:")
print("- BMI: Body Mass Index")
print("- Pulse Pressure: Systolic - Diastolic BP")
print("- Age Risk: Age > 50 (binary)")
print("- BP Risk: High blood pressure indicator")
print("- Lifestyle Risk: Smoking + Alcohol - Activity")

print("\nFinal dataset shape:", df.shape)
print("Features:", df.columns.tolist())

In [ ]:
# Identify features to drop based on low correlation and redundancy
features_to_drop = ['height', 'weight']  # We have BMI now
print(f"Dropping features: {features_to_drop}")
print("Reason: Redundant features (height, weight replaced by BMI)")

# Create final feature set
X = df.drop(['cardio'] + features_to_drop, axis=1)
y = df['cardio']

print("\nFinal feature set:")
print(X.columns.tolist())
print(f"Number of features: {X.shape[1]}")
print(f"Number of samples: {X.shape[0]}")

In [ ]:
# Data type conversion and standardization
print("Data types before conversion:")
print(X.dtypes)

# Ensure all features are numeric
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"\nNumeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# Convert any non-numeric to numeric if needed
for col in categorical_features:
    X[col] = pd.to_numeric(X[col], errors='coerce')

print("\nData types after conversion:")
print(X.dtypes)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nData standardized successfully!")
print(f"Mean of scaled training data: {np.mean(X_train_scaled):.6f}")
print(f"Std of scaled training data: {np.std(X_train_scaled):.6f}")

## B. Linear Regression with Gradient Descent

In [ ]:
# Custom Linear Regression with Gradient Descent
class LinearRegressionGD:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.costs_train = []
        self.costs_test = []
        
    def fit(self, X_train, y_train, X_test=None, y_test=None):
        # Initialize parameters
        n_samples, n_features = X_train.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Gradient descent
        for i in range(self.n_iterations):
            # Forward pass - training
            y_pred_train = np.dot(X_train, self.weights) + self.bias
            
            # Calculate cost - training
            cost_train = (1 / (2 * n_samples)) * np.sum((y_pred_train - y_train) ** 2)
            self.costs_train.append(cost_train)
            
            # Calculate cost - test (if provided)
            if X_test is not None and y_test is not None:
                y_pred_test = np.dot(X_test, self.weights) + self.bias
                cost_test = (1 / (2 * len(y_test))) * np.sum((y_pred_test - y_test) ** 2)
                self.costs_test.append(cost_test)
            
            # Calculate gradients
            dw = (1 / n_samples) * np.dot(X_train.T, (y_pred_train - y_train))
            db = (1 / n_samples) * np.sum(y_pred_train - y_train)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Train custom linear regression
lr_custom = LinearRegressionGD(learning_rate=0.01, n_iterations=1000)
lr_custom.fit(X_train_scaled, y_train, X_test_scaled, y_test)

print("Custom Linear Regression trained successfully!")

In [ ]:
# Plot loss curves
plt.figure(figsize=(12, 5))

# Loss curve
plt.subplot(1, 2, 1)
plt.plot(lr_custom.costs_train, label='Training Loss', color='blue')
plt.plot(lr_custom.costs_test, label='Test Loss', color='red')
plt.title('Training and Test Loss Curves')
plt.xlabel('Iterations')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.grid(True)

# Log scale for better visualization
plt.subplot(1, 2, 2)
plt.plot(lr_custom.costs_train, label='Training Loss', color='blue')
plt.plot(lr_custom.costs_test, label='Test Loss', color='red')
plt.title('Training and Test Loss Curves (Log Scale)')
plt.xlabel('Iterations')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.yscale('log')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Train scikit-learn Linear Regression for comparison
lr_sklearn = LinearRegression()
lr_sklearn.fit(X_train_scaled, y_train)

# Make predictions
y_pred_train_custom = lr_custom.predict(X_train_scaled)
y_pred_test_custom = lr_custom.predict(X_test_scaled)
y_pred_train_sklearn = lr_sklearn.predict(X_train_scaled)
y_pred_test_sklearn = lr_sklearn.predict(X_test_scaled)

# Evaluate models
print("Custom Linear Regression (Gradient Descent):")
print(f"Training MSE: {mean_squared_error(y_train, y_pred_train_custom):.6f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred_test_custom):.6f}")
print(f"Training R²: {r2_score(y_train, y_pred_train_custom):.6f}")
print(f"Test R²: {r2_score(y_test, y_pred_test_custom):.6f}")

print("\nScikit-learn Linear Regression:")
print(f"Training MSE: {mean_squared_error(y_train, y_pred_train_sklearn):.6f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred_test_sklearn):.6f}")
print(f"Training R²: {r2_score(y_train, y_pred_train_sklearn):.6f}")
print(f"Test R²: {r2_score(y_test, y_pred_test_sklearn):.6f}")

In [ ]:
# Scatter plots before and after regression
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Before regression - raw data
axes[0, 0].scatter(range(len(y_train)), y_train, alpha=0.6, color='blue', label='Actual')
axes[0, 0].set_title('Training Data - Before Regression')
axes[0, 0].set_xlabel('Sample Index')
axes[0, 0].set_ylabel('Cardiovascular Disease')
axes[0, 0].legend()

# After regression - training predictions
axes[0, 1].scatter(range(len(y_train)), y_train, alpha=0.6, color='blue', label='Actual')
axes[0, 1].plot(range(len(y_train)), y_pred_train_sklearn, color='red', linewidth=2, label='Predicted')
axes[0, 1].set_title('Training Data - After Linear Regression')
axes[0, 1].set_xlabel('Sample Index')
axes[0, 1].set_ylabel('Cardiovascular Disease')
axes[0, 1].legend()

# Test data - before
axes[1, 0].scatter(range(len(y_test)), y_test, alpha=0.6, color='green', label='Actual')
axes[1, 0].set_title('Test Data - Before Regression')
axes[1, 0].set_xlabel('Sample Index')
axes[1, 0].set_ylabel('Cardiovascular Disease')
axes[1, 0].legend()

# Test data - after regression
axes[1, 1].scatter(range(len(y_test)), y_test, alpha=0.6, color='green', label='Actual')
axes[1, 1].plot(range(len(y_test)), y_pred_test_sklearn, color='red', linewidth=2, label='Predicted')
axes[1, 1].set_title('Test Data - After Linear Regression')
axes[1, 1].set_xlabel('Sample Index')
axes[1, 1].set_ylabel('Cardiovascular Disease')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Actual vs Predicted scatter plots
plt.figure(figsize=(12, 5))

# Training data
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_pred_train_sklearn, alpha=0.6)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Training Data: Actual vs Predicted')
plt.grid(True)

# Test data
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_test_sklearn, alpha=0.6, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Test Data: Actual vs Predicted')
plt.grid(True)

plt.tight_layout()
plt.show()

## C. Model Comparison: Linear Regression vs Decision Trees vs Random Forest

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
}

# Train and evaluate models
results = {}
predictions = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    # Store predictions
    predictions[name] = {
        'train': y_pred_train,
        'test': y_pred_test
    }
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    results[name] = {
        'Train MSE': train_mse,
        'Test MSE': test_mse,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'Train MAE': train_mae,
        'Test MAE': test_mae,
        'CV R² Mean': cv_scores.mean(),
        'CV R² Std': cv_scores.std()
    }

print("\nModel training completed!")

In [ ]:
# Display results in a comprehensive table
results_df = pd.DataFrame(results).T
print("MODEL COMPARISON RESULTS:")
print("=" * 80)
print(results_df.round(6))

# Find best model
best_model_name = results_df['Test R²'].idxmax()
print(f"\n🏆 BEST PERFORMING MODEL: {best_model_name}")
print(f"Test R² Score: {results_df.loc[best_model_name, 'Test R²']:.6f}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# R² scores comparison
model_names = list(results.keys())
train_r2_scores = [results[name]['Train R²'] for name in model_names]
test_r2_scores = [results[name]['Test R²'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

axes[0, 0].bar(x - width/2, train_r2_scores, width, label='Train R²', alpha=0.8)
axes[0, 0].bar(x + width/2, test_r2_scores, width, label='Test R²', alpha=0.8)
axes[0, 0].set_xlabel('Models')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_title('Model Performance Comparison (R² Score)')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(model_names, rotation=45)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# MSE comparison
train_mse_scores = [results[name]['Train MSE'] for name in model_names]
test_mse_scores = [results[name]['Test MSE'] for name in model_names]

axes[0, 1].bar(x - width/2, train_mse_scores, width, label='Train MSE', alpha=0.8)
axes[0, 1].bar(x + width/2, test_mse_scores, width, label='Test MSE', alpha=0.8)
axes[0, 1].set_xlabel('Models')
axes[0, 1].set_ylabel('Mean Squared Error')
axes[0, 1].set_title('Model Performance Comparison (MSE)')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(model_names, rotation=45)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Cross-validation scores
cv_means = [results[name]['CV R² Mean'] for name in model_names]
cv_stds = [results[name]['CV R² Std'] for name in model_names]

axes[1, 0].bar(model_names, cv_means, yerr=cv_stds, capsize=5, alpha=0.8)
axes[1, 0].set_xlabel('Models')
axes[1, 0].set_ylabel('Cross-Validation R² Score')
axes[1, 0].set_title('Cross-Validation Performance')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Feature importance for Random Forest
if 'Random Forest' in models:
    rf_model = models['Random Forest']
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=True)
    
    axes[1, 1].barh(feature_importance['feature'], feature_importance['importance'])
    axes[1, 1].set_xlabel('Feature Importance')
    axes[1, 1].set_title('Random Forest Feature Importance')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis for all models
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (name, model) in enumerate(models.items()):
    y_pred = predictions[name]['test']
    residuals = y_test - y_pred
    
    axes[i].scatter(y_pred, residuals, alpha=0.6)
    axes[i].axhline(y=0, color='red', linestyle='--')
    axes[i].set_xlabel('Predicted Values')
    axes[i].set_ylabel('Residuals')
    axes[i].set_title(f'{name} - Residual Plot')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Save the best model and scaler
best_model = models[best_model_name]

# Save model and scaler
with open('best_cardio_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save feature names
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(X.columns.tolist(), f)

print(f"✅ Best model ({best_model_name}) saved as 'best_cardio_model.pkl'")
print("✅ Scaler saved as 'scaler.pkl'")
print("✅ Feature names saved as 'feature_names.pkl'")

# Display final summary
print("\n" + "="*80)
print("CARDIOVASCULAR DISEASE PREDICTION MODEL - SUMMARY")
print("="*80)
print(f"🎯 Mission: Predict cardiovascular disease to ensure healthier workforce")
print(f"📊 Dataset: {df.shape[0]} samples, {X.shape[1]} features")
print(f"🏆 Best Model: {best_model_name}")
print(f"📈 Test R² Score: {results[best_model_name]['Test R²']:.4f}")
print(f"📉 Test MSE: {results[best_model_name]['Test MSE']:.6f}")
print(f"🎯 Cross-Validation R²: {results[best_model_name]['CV R² Mean']:.4f} ± {results[best_model_name]['CV R² Std']:.4f}")
print("="*80)